In [1]:
from datasets import load_dataset

dataset = load_dataset("./", data_dir="../data/Brain/train/",split="train")

/home/pooria99/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 3092/3092 [00:00<00:00, 141828.39it/s]


In [2]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128  # the generated image resolution
    train_batch_size = 16
    eval_batch_size = 16  # how many images to sample during evaluation
    num_epochs = 100
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "results/"  # the model name locally and on the HF Hub

    push_to_hub = False  # whether to upload the saved model to the HF Hub
    # hub_model_id = "<your-username>/<my-awesome-model>"  # the name of the repository to create on the HF Hub
    # hub_private_repo = False
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0


config = TrainingConfig()

In [3]:
from torchvision import transforms

preprocess = transforms.Compose(
    [
        transforms.Resize((config.image_size, config.image_size)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

In [4]:
def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}


dataset.set_transform(transform)

In [5]:
import torch

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

In [6]:
import diffusers
print(dir(diffusers))


['AltDiffusionImg2ImgPipeline', 'AltDiffusionPipeline', 'AnimateDiffPipeline', 'AsymmetricAutoencoderKL', 'AudioDiffusionPipeline', 'AudioLDM2Pipeline', 'AudioLDM2ProjectionModel', 'AudioLDM2UNet2DConditionModel', 'AudioLDMPipeline', 'AudioPipelineOutput', 'AutoPipelineForImage2Image', 'AutoPipelineForInpainting', 'AutoPipelineForText2Image', 'AutoencoderKL', 'AutoencoderTiny', 'BlipDiffusionControlNetPipeline', 'BlipDiffusionPipeline', 'CLIPImageProjection', 'CMStochasticIterativeScheduler', 'ConfigMixin', 'ConsistencyDecoderVAE', 'ConsistencyModelPipeline', 'ControlNetModel', 'CycleDiffusionPipeline', 'DDIMInverseScheduler', 'DDIMParallelScheduler', 'DDIMPipeline', 'DDIMScheduler', 'DDPMParallelScheduler', 'DDPMPipeline', 'DDPMScheduler', 'DDPMWuerstchenScheduler', 'DEISMultistepScheduler', 'DPMSolverMultistepInverseScheduler', 'DPMSolverMultistepScheduler', 'DPMSolverSDEScheduler', 'DPMSolverSinglestepScheduler', 'DanceDiffusionPipeline', 'DiTPipeline', 'DiffusionPipeline', 'DummyOb

In [7]:
from diffusers import DiffusionPipeline
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")

text_encoder/config.json: 100%|██████████| 565/565 [00:00<00:00, 169kB/s]



tokenizer/tokenizer_config.json: 100%|██████████| 737/737 [00:00<00:00, 766kB/s]
tokenizer/merges.txt: 100%|██████████| 525k/525k [00:00<00:00, 8.54MB/s]






tokenizer_2/special_tokens_map.json: 100%|██████████| 460/460 [00:00<00:00, 2.14MB/s]



tokenizer_2/tokenizer_config.json: 100%|██████████| 725/725 [00:00<00:00, 2.79MB/s]



unet/config.json: 100%|██████████| 1.68k/1.68k [00:00<00:00, 5.86MB/s]








vae/config.json: 100%|██████████| 642/642 [00:00<00:00, 1.20MB/s]






tokenizer/vocab.json: 100%|██████████| 1.06M/1.06M [00:00<00:00, 3.25MB/s]






































tokenizer_2/vocab.json: 100%|██████████| 1.06M/1.06M [00:00<00:00, 1.43MB/s]























































































diffusion_pytorch_model.fp16.safetensors: 100%|██████████| 167M/167M [00:03<00:00, 47.3MB/s]


diffusion_pytorch_model.fp16.safetensors: 100%|██████████| 167M/

In [10]:
print(pipe)

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.23.1",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "force_zeros_for_empty_prompt": true,
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

